#  Project 2: Gesture recognition ~ LINFO2275

## Baseline for gesture recognition

### Preliminaries

In [ ]:
# Librairies to use
import os 
import linecache
import pandas as pd
import numpy as np
import math
import scipy.spatial.distance as dist
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import natsort
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# The path to the folder containing the data
root1 = "E:/DOCUMENTS/CIVILE/BELGIQUE/MASTER UCL/LSBA_2021/DATS2M/BLOC 1/QUADRI 2"
root2 = "/LINFO2275_DATA MINING & DECISION MAKING/PROJET/GESTURE RECOGNITION/WORK/Domain01"
chemin = root1 + root2

# Change or setting up the directory 
os.chdir(chemin)

### Data preprocessing

In [ ]:
# the idea here is to transform the data sets to a numpy array structure to easily 
# apply dynamic time warping the the numpy array

# List of the sketch : 1,2,3...
sketch_draw = []

# list to store the data identification
sketch_id = []

# list to store the data on array structure
sketch_data = []

# list of the user id
sketch_user = []

# Reading the file in my directory
for file in natsort.natsorted(os.listdir()): #os.listdir():
    
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        
        # Getting the class of the sketch : 
        class_id = linecache.getline(file,2).split("=")[1].split()[0]
        
        # Getting the number drawn
        sketch_draw.append(class_id)
        
        # Getting the user id: 
        user_id = linecache.getline(file,3).split("=")[1].split()[0]
        
        # Getting the concerned user
        sketch_user.append(user_id)
        
        # Creating an identifiant 
        combi = "User_"+ user_id + "_class_" + class_id
        
        # Adding the created identifiant to the data id list
        sketch_id.append(combi)
        
        # Reading the txt files
        data = pd.read_csv(file, skiprows = 5, names = ["x","y","z","t"], usecols = ["x","y","z"])
        
        # Appending the data to the data list after converting the dataframe to numpy array structure 
        sketch_data.append(data.to_numpy())

### Distance matrix

In [ ]:
# Distance matrix of the diqtance between points of two sketch

# As the sketch is a 3D coordinate, the distance metric to use is the euclidean distance metric

def distance(Point_A, Point_B):
    """
    Argument : Vector of points coordinate (data frame of sketch in numpy array format)
    
    Return : a matrix of the distance each point of the two dataframe.
                the shape of this matrix is the the length of the arrays
    """
    # Defining number of rows 
    N = Point_A.shape[0]
    
    # Defining number of columns
    M = Point_B.shape[0]
    
    # Defining the matrix
    dist_mat = np.zeros((N, M))
    
    # Filling up the matrix
    for i in range(N):
        for j in range(M):
            
            # Euclidena distance calculation
            dist_mat[i, j] = np.linalg.norm(Point_A[i]-Point_B[j])
    return dist_mat

### Dynamic time warpin algorithm function 

In [ ]:
def dp(dist_mat):
    """
    Argument: dist_mat is the distance matrix of the sketch
    
    Return : 
    
    Find minimum-cost path through matrix `dist_mat` using dynamic programming.

    The cost of a path is defined as the sum of the matrix entries on that
    path. See the following for details of the algorithm:

    - http://en.wikipedia.org/wiki/Dynamic_time_warping
    - https://www.ee.columbia.edu/~dpwe/resources/matlab/dtw/dp.m

    The notation in the first reference was followed, while Dan Ellis's code
    (second reference) was used to check for correctness. Returns a list of
    path indices and the cost matrix.
    """

    N, M = dist_mat.shape
    
    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],      # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)

### Alignment cost  of two sketches

In [ ]:
# Function returning the cost of the alignment of two sketch
def align_cost(sketch1, sketch2):
    """ 
    Argument: the two sketchs to compare in array structure
    
    Return: the cost of the alignment
    """
    return round(dp(distance(sketch1,sketch2))[1][len(sketch1) - 1, len(sketch2) - 1],4)

### 10-most align sketches with the specified sketch

In [ ]:
# Function computing the most align sketch
def most_align(sketch):
    """
    Arguments : the sketch you want to get it recognized pattern
    
    Return: this function returns two values:
    
        - a sorted list of the 10 smaller alignment cost with all other sketch
        - and the list of the 10 patterns (class) associated to the list of alignment list 
    """
    # list containing the sketch relatively to other sketch in the dataframe
    all_cost = []
    
    # List of the 8-most aligned with the main sketch
    align = []
    
    # List of the drawn relative to
    drawn_relative = []
    
    # Getting the cost and appending it to the cost list 
    for val in sketch_data:
        all_cost.append(align_cost(val, sketch))
    
    # Getting the index of the cost
    index_sorted = np.argsort(all_cost)
    
    # index of 10-most align sketch  
    most_10 = index_sorted[1:11]
    
    # Getting the corresponding sketch_id
    for i in most_10: 
        align.append(sketch_id[i])
        drawn_relative.append(sketch_draw[i])
    
    return align, drawn_relative

### Algorithm for most align aligned sketch and selection of recognized pattern 

In [ ]:
# User i sketches identification

def recognized(id_user):
    
    # lists to retain the index of the data to be used for comparison against the effective data
    comparing_list = []
    effective_list = []
    
    # Go througth the sketch_user list to get the index of the concerned data
    for k in range(len(sketch_user)):
        
        if int(sketch_user[k]) != id_user:
            comparing_list.append(k)
        else :
            effective_list.append(k)
    
        
    # the recognized pattern
    recognized_pattern = []
    
    # concerned pattern
    concerned_pattern = []
    
    # Loop throughtout the effective list 
    for i in effective_list[::10]:
        
        # list to store the alignment cost 
        all_cost = []
        
        # Loop throughout the comparing_list
        for j in comparing_list:
                        
            # Add the alignment cost between i and each j to list of cost 
            all_cost.append(align_cost(sketch_data[i],sketch_data[j]))
        
        # take the index of the order the cost list 
        index_sorted = np.argsort(all_cost)
        
        # appending the recognized pattern to the list
        recognized_pattern.append(sketch_draw[index_sorted[1]])
        
        # real pattern 
        concerned_pattern.append(sketch_draw[i])
        
    return [concerned_pattern,recognized_pattern]

## Gesture-based system analysis

### Kernel matrix 

In [ ]:
noyau = []
for i in range(len(sketch_data)):
    noyau1 = []
    for j in range(len(sketch_data)):
        noyau1.append(round(sum(np.matrix.diagonal(distance(sketch_data[i],sketch_data[j]))),2))
    noyau.append(noyau1)

noyau